# Implementing k-NN for Wikipedia Article Segmentation

##### This is based on assignment in the Machine Learning specialization by University of Washington on Coursera
In this notebook I have implemented two different types of distance measure (a) Euclidean distance (b) Cosine similarity. 

Further, I have implemented two different types of document representation (a) Word count (b) Term frequency-Inverse Document frequency. Both these methods have been implemented through brute force method. In another notebook elsewhere I have implemented KD-Tree for search space segmentation.  

The documents are provided as text in the python data frame. 

In [98]:
import numpy as ny 
import pandas as pd
import math 
import operator

In [99]:
people_wiki = pd.read_csv("people_wiki.csv")

#### Calculating Euclidean distance between Barack Obama and other articles using "word count" through Brute Force approach

In [100]:
text=people_wiki['text']  ## to get the cell value from a dataframe

In [101]:
word_count= []
for i in range(0,len(text)):
    text_words=text[i].split()
    word_list = {}
    for words in text_words:
        if words in word_list:
            word_list[words]=word_list[words]+1
        else:
            word_list[words]=1
    word_count.append(word_list)

In [102]:
people_wiki['word_count']=word_count

In [20]:
BO=people_wiki[people_wiki['name']=='Barack Obama']

In [21]:
temp=people_wiki.head(100)

In [22]:
k=10    # k is the number of nearest neigbour in k-NN
list_of_knn={}
for names in temp['name']:
    dist=0
    TR=temp[temp['name']==names]
    for xq_keys in BO.iloc[0]['word_count'].keys():
        xq=BO.iloc[0]['word_count'][xq_keys]
        if xq_keys in TR.iloc[0]['word_count'].keys():
            xi=TR.iloc[0]['word_count'][xq_keys]
            dist=dist+(xq-xi)*(xq-xi)
        else:
            dist=dist+xq*xq
    list_of_knn[TR.iloc[0]['name']]=math.sqrt(dist)
    sorted_list_of_knn = sorted(iter(list_of_knn.items()), key=operator.itemgetter(1))
    knn=sorted_list_of_knn[0:k]    

#### Calculating Euclidean distance between Barack Obama and other articles using "TF-IDF" through Brute Force approach

In [25]:
BO=people_wiki[people_wiki['name']=='Barack Obama']

In [221]:
temp_TFIDF=people_wiki[people_wiki['name'].isin(['Barack Obama', 'Joe Biden', 'Jeff Sessions', 'Jesse Lee (politician)','Samantha Power'])]

In [222]:
words_idf = {} 
for names in temp_TFIDF['name']:
    curr=temp_TFIDF[temp_TFIDF['name']==names]
    for words in curr.iloc[0]['word_count'].keys():
        for i in range(0,len(temp_TFIDF)):
            if words in temp_TFIDF.iloc[i]['word_count'].keys():
                if words not in words_idf.keys():
                    words_idf[words]=1
                else:
                    words_idf[words]=words_idf[words]+1              
        

In [223]:
tf_idf_list=[]
for names in temp_TFIDF['name']:
    tf_idf={}
    curr=temp_TFIDF[temp_TFIDF['name']==names]
    for words in curr.iloc[0]['word_count'].keys():
        tf_idf[words]=math.log((curr.iloc[0]['word_count'][words])/(1+words_idf[words]))
    tf_idf_list.append(tf_idf)
temp_TFIDF['tf_idf']=tf_idf_list

In [93]:
BO=temp_TFIDF[temp_TFIDF['name']=='Barack Obama']

In [94]:
k=2    # k is the number of nearest neigbour in k-NN
list_of_knn_tf={}
for names in temp_TFIDF['name']:
    dist=0
    TR=temp_TFIDF[temp_TFIDF['name']==names]
    for xq_keys in BO.iloc[0]['tf_idf'].keys():
        xq=BO.iloc[0]['tf_idf'][xq_keys]
        if xq_keys in TR.iloc[0]['tf_idf'].keys():
            xi=TR.iloc[0]['tf_idf'][xq_keys]
            dist=dist+(xq-xi)*(xq-xi)
        else:
            dist=dist+xq*xq
    list_of_knn_tf[TR.iloc[0]['name']]=math.sqrt(dist)
    sorted_list_of_knn_tf = sorted(iter(list_of_knn_tf.items()), key=operator.itemgetter(1))
    knn_tf=sorted_list_of_knn_tf[0:k]

#### Calculating Cosine Similarity between Barack Obama and other articles using "TF-IDF" through Brute Force approach

In [224]:
BO=temp_TFIDF[temp_TFIDF['name']=='Barack Obama']

In [225]:
temp_cosine=temp_TFIDF[temp_TFIDF['name'].isin(['Barack Obama', 'Joe Biden', 'Jeff Sessions', 'Jesse Lee (politician)','Samantha Power'])]

In [226]:
k=2    # k is the number of nearest neigbour in k-NN
list_of_knn_cosine={}
for names in temp_cosine['name']:
    xq = []
    xi = []
    TR=temp_cosine[temp_cosine['name']==names]
    for keys in BO.iloc[0]['tf_idf'].keys():
        if keys in TR.iloc[0]['tf_idf'].keys():
            xq.append(BO.iloc[0]['tf_idf'][keys])
            xi.append(TR.iloc[0]['tf_idf'][keys])
    x_num=0
    xq_den=0
    xi_den=0
    for i in range(1,len(xq)):
        x_num=x_num+xq[i]*xi[i]
        xq_den=xq_den+xq[i]*xq[i]
        xi_den=xi_den+xi[i]*xi[i]
    list_of_knn_cosine[names]=1-(x_num/(math.sqrt(xq_den)*math.sqrt(xi_den)))